In [3]:
from pgpy.constants import PubKeyAlgorithm, KeyFlags, HashAlgorithm, SymmetricKeyAlgorithm, CompressionAlgorithm
import pgpy
import os
import glob
from secrets import token_hex

In [4]:
def key_generation(user_name,passphrase,salt):
    key = pgpy.PGPKey.new(PubKeyAlgorithm.RSAEncryptOrSign, 2048)
    uid = pgpy.PGPUID.new(user_name)

    key.add_uid(uid, usage={KeyFlags.Sign, KeyFlags.EncryptCommunications, KeyFlags.EncryptStorage},
            hashes=[HashAlgorithm.SHA256, HashAlgorithm.SHA384, HashAlgorithm.SHA512, HashAlgorithm.SHA224],
            ciphers=[SymmetricKeyAlgorithm.AES256, SymmetricKeyAlgorithm.AES192, SymmetricKeyAlgorithm.AES128],
            compression=[CompressionAlgorithm.ZLIB, CompressionAlgorithm.BZ2, CompressionAlgorithm.ZIP, CompressionAlgorithm.Uncompressed])
    key.protect(passphrase+salt, SymmetricKeyAlgorithm.AES256, HashAlgorithm.SHA256)
    return key

In [5]:
# message is in string given by User
# pub_key is fetched from Database i.e Receivers public Key
def encryption(pub_key,message):
    pub_key_enc_pgpy, _ = pgpy.PGPKey.from_blob(pub_key)
    text_message = pgpy.PGPMessage.new(message)
    encrypted_message = pub_key_enc_pgpy.encrypt(text_message)
    return encrypted_message

In [6]:
# sec_key is Private key stored on local device
# enc_message is encrypted message
def decryption(sec_key,enc_message,passphrase,salt):
    temp_pgpy = pgpy.PGPKey()
    temp_pgpy.parse(sec_key)
    message = pgpy.PGPMessage()
    message.parse(enc_message)
    try:
        with temp_pgpy.unlock(passphrase+salt):
            decrypted_message = temp_pgpy.decrypt(message)
            return decrypted_message.message
    except:
        print("Wrong passphrase")

In [7]:
# sec_key is key stored at local machine i.e private key
# message is string
def sign(sec_key,message,passphrase,salt):
    temp_pgpy = pgpy.PGPKey()
    temp_pgpy.parse(sec_key)
    try:
        with temp_pgpy.unlock(passphrase+salt):
            message = pgpy.PGPMessage.new(message)
            signature = temp_pgpy.sign(message)
            return signature
    except:
        print("Wrong passphrase")

In [8]:
# pub_key is key fetched from DB
# message is signed string message
def verify(pub_key,message,signature):
    pub_key, _ = pgpy.PGPKey.from_blob(pub_key)
    temp_pgpy = pgpy.PGPSignature()
    temp_pgpy.parse(signature)
    if pub_key.verify(message,temp_pgpy):
        return True
    else:
        return False


In [9]:
def get_salt():
    return str(token_hex(16))

In [10]:
def combine(enc,sign):
    return (str(enc)+str(sign))

In [11]:
def separate_enc_sign(combined_msg):
    index_of_sign = combined_msg.find("-----BEGIN PGP SIGNATURE-----")
    extracted_msg = combined_msg[0:index_of_sign]
    extracted_sign = combined_msg[index_of_sign:len(combined_msg)]
    return extracted_msg,extracted_sign

In [25]:
# INFO=======================================
# store the OUTPUT of key_generation function on local machine as private key
# Get public key using key.pubkey method and store it on DB

In [26]:
# Sender Side
salt1 = get_salt()
sender_key = key_generation('sender',"SecretPassphrase1",salt1)
# print(sender_key.pubkey)

In [27]:
#Receiver Side
salt2=get_salt()
receiver_key = key_generation('receiver',"SecretPassphrase2",salt2)
# print(receiver_key.pubkey)

In [28]:
sender_sec_str = str(sender_key) # sec_str is key from local machine
sender_pub_str = str(sender_key.pubkey) # pub_str is key from DB

In [29]:
receiver_sec_str = str(receiver_key) # sec_str is key from local machine
receiver_pub_str = str(receiver_key.pubkey) # pub_str is key from DB

In [30]:
#Sender Side
enc = encryption(receiver_pub_str,"This is Time Pass Message!")
# print(enc)
print(type(enc))
# print("encrypted? " + str(enc.is_encrypted))
# print("Signed? " + str(enc.is_signed))

<class 'pgpy.pgp.PGPMessage'>


In [31]:
#Sender Side
signature = sign(sender_sec_str,str(enc),"SecretPassphrase1",salt1)
print(signature)
print(type(signature))

-----BEGIN PGP SIGNATURE-----

wsBcBAABCAAGBQJepWcnAAoJEOs2pg/lu1Ex4EAH/3HZ1fgUoGixu4VaZXAti3Nl
TxlAta2YEJ0PagocDoU0jR2hRYBNMsnySlZzj0Nt7euBNiEcoDmg4/AFmfPnlXm6
u05/pLYDtYFMz3mQEp50LVke6eZNIdv6QMg9TF12YtQBDBMj+5jQToUHSplfLYEX
Do+k8S/Wl/MqpfuJb4I8vs10rnWaulYxmEHxRkwzPYIlKyGdCCVBfOdnJtZKpnBH
ZpGzyw/WbktZaI4Kv5NQX6e7PJU1T0GGr6LgVDrJkXtj4TGCJvYXjrZEua3LAfPz
hjzo5soTD30Xoz81cDQOsgpl5dGGvGPfmmAeldQVkMrkrFuGFg/lluo/E68oU1g=
=Ke91
-----END PGP SIGNATURE-----

<class 'pgpy.pgp.PGPSignature'>


In [32]:
#sender side
combined_msg = combine(enc,signature)
print(combined_msg)

-----BEGIN PGP MESSAGE-----

wcBMA9PPvP2FNdMAAQgAkA2S3pQ2Z1kGY2McM5IFAatQ/RcQRimK/OtSkpnrToJ9
XXYrO+iHRixrd4wcoVg4lH+Lwf++I5E0xGA/UuXDrRAsPc0y8YqIGA/lrDkPb+c/
+01eBVTwq/SyuizKMKSrfocGHVDi4J3wUTEohr7ezJTHqc7ZEHuMX0UPHD89OX7f
g6XvSxVg3zIUQWWPcn4SQxudYm2ygazUQkqEY0aHGEHXrOBon+RiI+WzNAkhkgVt
3MEzs86HdPuU3Tu6ug2sjzAOb0cG85RNyBlvWUCpUIbuwUgyndZB4zT9lP0OHQ9n
Ka0IZDIkdV/IIxOl4rZG1LZKwtXhXiD3HG4gokpgcNJOAQhAcTyDvmG7gjqPNnNN
Y4JcxRLcfFWyuc6czBqSzFWOmwFv+rO7XUK5CZTAv0dg5vNqS8k0D4/dorxXfHBa
5K7FiQ16MNcDsOT0U6mk
=VLar
-----END PGP MESSAGE-----
-----BEGIN PGP SIGNATURE-----

wsBcBAABCAAGBQJepWcnAAoJEOs2pg/lu1Ex4EAH/3HZ1fgUoGixu4VaZXAti3Nl
TxlAta2YEJ0PagocDoU0jR2hRYBNMsnySlZzj0Nt7euBNiEcoDmg4/AFmfPnlXm6
u05/pLYDtYFMz3mQEp50LVke6eZNIdv6QMg9TF12YtQBDBMj+5jQToUHSplfLYEX
Do+k8S/Wl/MqpfuJb4I8vs10rnWaulYxmEHxRkwzPYIlKyGdCCVBfOdnJtZKpnBH
ZpGzyw/WbktZaI4Kv5NQX6e7PJU1T0GGr6LgVDrJkXtj4TGCJvYXjrZEua3LAfPz
hjzo5soTD30Xoz81cDQOsgpl5dGGvGPfmmAeldQVkMrkrFuGFg/lluo/E68oU1g=
=Ke91
-----END PGP SIGNATURE-----



In [33]:
#Receiver side
extracted_msg,extracted_sign = separate_enc_sign(combined_msg)
# print(extracted_msg)

In [34]:
# print(extracted_sign)

In [35]:
#Receiver Side
print(verify(sender_pub_str,str(extracted_msg),str(extracted_sign)))

True


In [36]:
#Receiver Side
print(decryption(receiver_sec_str,extracted_msg,"SecretPassphrase2",salt2))

This is Time Pass Message!
